<a href="https://colab.research.google.com/github/biruk50/Medium_articles/blob/main/test_depth_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project requires Python 3.7 or above:



In [ ]:
import sys

assert sys.version_info >= (3, 7)

In [1]:
import time
import tracemalloc
from collections import deque

#this is for the modififed DFS
class Stack:
    def __init__(self):
        self.items = deque()

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def pop_left(self):
        return self.items.popleft()

    def is_empty(self):
        return len(self.items) == 0

In [3]:
# Modified DFS
def modified_dfs(first):
    search_stack = Stack()
    search_stack.push(first)
    searched = set()
    parent_map = {first: None}
    checked_order = []

    while not search_stack.is_empty():
        # Check if the top element of the stack has no neighbors
        if not graph[search_stack.items[-1]]:
            # Pop the node with no neighbors
            person = search_stack.pop()

            leftmost = search_stack.pop_left()
            search_stack.push(leftmost)
        else:
            # Pop the top element normally
            person = search_stack.pop()

        if person not in searched:
            checked_order.append(person)
            if target(person):
                # Construct the list of parent nodes
                path = []
                while person is not None:
                    path.append(person)
                    person = parent_map[person]
                return True, path[::-1], checked_order
            searched.add(person)

            # Push neighbors to the stack
            for neighbor in graph[person]:
                if neighbor not in searched:
                    search_stack.push(neighbor)
                    parent_map[neighbor] = person

    return False, [], checked_order

# Regular DFS
def dfs(first):
    search_stack = [first]
    searched = set()
    parent_map = {first: None}
    checked_order = []

    while search_stack:
        person = search_stack.pop()

        if person not in searched:
            checked_order.append(person)
            if target(person):
                path = []
                while person is not None:
                    path.append(person)
                    person = parent_map[person]
                return True, path[::-1], checked_order
            searched.add(person)
            for neighbor in graph.get(person, []):
                if neighbor not in searched:
                    search_stack.append(neighbor)
                    parent_map[neighbor] = person

    return False, [], checked_order

# BFS
def bfs(first):
    search_queue = deque([first])
    searched = set()
    parent_map = {first: None}
    checked_order = []

    while search_queue:
        person = search_queue.popleft()

        if person not in searched:
            checked_order.append(person)
            if target(person):
                path = []
                while person is not None:
                    path.append(person)
                    person = parent_map[person]
                return True, path[::-1], checked_order
            searched.add(person)
            for neighbor in graph.get(person, []):
                if neighbor not in searched:
                    search_queue.append(neighbor)
                    parent_map[neighbor] = person

    return False, [], checked_order

# IDDFS
def iddfs(root):
    def dls(node, depth, visited):
        if depth == 0:
            if target(node):
                return True, [node]
            return False, []
        if depth > 0:
            for neighbor in graph.get(node, []):
                if neighbor not in visited:
                    visited.add(neighbor)
                    found, path = dls(neighbor, depth - 1, visited)
                    if found:
                        return True, [node] + path
        return False, []

    depth = 0
    while True:
        visited = {root}
        found, path = dls(root, depth, visited)
        if found:
            return True, path, list(visited)
        depth += 1

# Performance measurement
def measure_performance(search_function, first):
    tracemalloc.start()
    start_time = time.time()
    found, path, checked_order = search_function(first)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    return {
        'time': end_time - start_time,
        'memory': peak,
        'found': found,
        'path': path,
        'checked_order': checked_order
    }

# Graph and target definition
graph = {
    "A": ["B", "C", "D", "E"],    "B": ["F", "G"],
    "C": ["H", "I"],   "D": [],    "E": ["J", "K"],
    "F": ["L", "M"],   "G": [],    "H": ["N", "O"],
    "I": [],    "J": [],     "K": ["P", "Q"],
    "L": [],    "M": [],     "N": ["R"],     "O": [],
    "P": [],    "Q": [],     "R": []
}

def target(name):
    return name == "P"

# Measure performance
first_node = "A"
results = {
    'modified_dfs': measure_performance(modified_dfs, first_node),
    'dfs': measure_performance(dfs, first_node),
    'bfs': measure_performance(bfs, first_node),
    'iddfs': measure_performance(iddfs, first_node)
}

for algo, result in results.items():
    print(f"Algorithm: {algo}")
    print(f"Time taken: {result['time']} seconds")
    print(f"Memory used: {result['memory']} bytes")
    print(f"Found: {result['found']}")
    print(f"Path to target: {result['path']}")
    print(f"Checked nodes order: {result['checked_order']}")
    print()


Algorithm: modified_dfs
Time taken: 0.00010156631469726562 seconds
Memory used: 2784 bytes
Found: True
Path to target: ['A', 'E', 'K', 'P']
Checked nodes order: ['A', 'E', 'K', 'Q', 'B', 'G', 'C', 'I', 'D', 'J', 'P']

Algorithm: dfs
Time taken: 2.7418136596679688e-05 seconds
Memory used: 1184 bytes
Found: True
Path to target: ['A', 'E', 'K', 'P']
Checked nodes order: ['A', 'E', 'K', 'Q', 'P']

Algorithm: bfs
Time taken: 3.123283386230469e-05 seconds
Memory used: 2288 bytes
Found: True
Path to target: ['A', 'E', 'K', 'P']
Checked nodes order: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']

Algorithm: iddfs
Time taken: 7.104873657226562e-05 seconds
Memory used: 3760 bytes
Found: True
Path to target: ['A', 'E', 'K', 'P']
Checked nodes order: ['L', 'B', 'D', 'G', 'H', 'A', 'F', 'P', 'J', 'M', 'N', 'O', 'I', 'K', 'E', 'C']

